In [1]:
import math

import numpy as np

import pandas as pd



def group(df,att,class_att,rows):

    count={}

    rownums={}

    for r in rows:

        val=df.loc[r,att]

        cls=df.loc[r,class_att]

        if val not in count:

            count[val]={}

            count[val]['TOTAL']=0

            rownums[val]=[]

        if cls not in count[val]:

            count[val][cls]=0

        count[val][cls]+=1

        count[val]['TOTAL']+=1

        rownums[val].append(r)

    return(count,rownums)

    

    

def find_entropy(df,att,class_att,rows):

    entropy=0

    (count,rownums)=group(df,att,class_att,rows)

    for val in count:

        temp=0

        total=count[val]['TOTAL']

        for cls in count[val]:

            if cls!='TOTAL':

                v=count[val][cls]

                temp+=-v/total*math.log(v/total)

        entropy+=temp*(total/len(rows))

    return(entropy,rownums)

    

def createTree(df,avail_col,rows,class_att):

    if len(df.loc[rows,class_att].unique())==1:

        return df.loc[rows[0],class_att]

    elif len(avail_col)==0:

        return DEFAULT

    min_entropy=np.inf

    for attribute in avail_col:

        (entropy,rownums)=find_entropy(df,attribute,class_att,rows)

        if entropy<min_entropy :

            min_entropy=entropy

            min_att=attribute

            min_rownums=rownums

    avail_col.remove(min_att)

    for val in min_rownums:

        min_rownums[val]=createTree(df,avail_col,min_rownums[val],class_att)

    return(min_att,min_rownums)









def printTree(tree,rule):

    if isinstance(tree,str):

        print (rule+' then '+tree)

    else:

        for val in tree[1]:

            printTree(tree[1][val],rule+','+tree[0]+'='+val)

            



def predict(tree,test):

    for r in range(len(test)):

        node=tree

        while not isinstance (node,str):

            val=test.loc[r,node[0]]

            if val in node[1]:

                node=node[1][val]

            else:

                prediction=DEFAULT

                break

        if isinstance(node,str):

            prediction=node

        print(prediction)                

    

    

df=pd.read_csv("/home/ishaan/weatherFull.csv")



DEFAULT=df.iloc[0,-1]   

tree=createTree(df,list(df.columns[:-1]),range(len(df)),df.columns[-1])

print(tree)

test=pd.read_csv("test.csv")

tst=predict(tree,test)

print(list(test.loc[:]))

print(tst)

('Outlook', {'Sunny': ('Temperature', {'Hot': 'No', 'Mild': ('Wind', {'Weak': 'No', 'Strong': 'Yes'}), 'Cool': 'Yes'}), 'Overcast': 'Yes', 'Rain': ('Humidity', {'High': 'No', 'Normal': 'No'})})


FileNotFoundError: File b'test.csv' does not exist